In [1]:
include("src\\building_tree.jl")
include("src\\utilities.jl")
include("src\\merge.jl")
include("src\\building_tree.jl")
# include("src\\struct\\new_cluster.jl")
using Debugger
using DataFrames, Random
using DataStructures
using StatsBase
using Statistics
using LinearAlgebra
# using Plots

In [8]:
dataSetName = "iris"
include("./data/" * dataSetName * ".txt")
# Ramener chaque caractéristique sur [0, 1]
reducedX = Matrix{Float64}(X)
for j in 1:size(X, 2)
    reducedX[:, j] .-= minimum(X[:, j])
    reducedX[:, j] ./= maximum(X[:, j])
end

train, test = train_test_indexes(length(Y))
X_train = reducedX[train,:]
Y_train = Y[train]
X_test = reducedX[test,:]
Y_test = Y[test]
classes = unique(Y)

println(" (train size ", size(X_train, 1), ", test size ", size(X_test, 1), ", ", size(X_train, 2), ", features count: ", size(X_train, 2), ")")

# Temps limite de la
time_limit = 10
D = 3
x = X_train
y = Y_train

 (train size 120, test size 30, 4, features count: 4)


120-element Vector{Int64}:
 3
 1
 3
 1
 1
 1
 1
 1
 2
 1
 ⋮
 3
 1
 2
 3
 1
 1
 1
 2
 1

In [23]:
n = length(Y_train)
m = length(x[1,:])
clusters = Vector{Cluster}([])
for dataId in 1:size(x,1)
    push!(clusters, Cluster(dataId, x,y))
end
clusterId = collect(1:n) #On obtient un vecteur 1,2..., qui correspond pour chaque cluster à son clusterId
distances = Vector{Distance}([])
for id1 in 1:n-1
    for id2 in id1+1:n
        push!(distances, Distance(id1, id2, x))
    end
end
sort!(distances, by = v ->v.distance)
remainingClusters=n
distanceId = 1
n_epochs = 1
c1_bis = Nothing
c2_bis = Nothing
i = 1
a=i
empty_clusters = Vector{Int64}([])
clusters_ids =Vector{Int64}([])
for i in  1:n
    push!(clusters_ids,i)
end

In [24]:
distanceId = 4
max_elements_small_classes = 5
num_clusters = 10
distance = distances[distanceId]
cId1 =clusterId[distance.ids[1]]
cId2 = clusterId[distance.ids[2]]
if cId1 != cId2
    c1 = clusters[cId1]
    c2 = clusters[cId2]
end

Cluster([84], [0.30555555555555564, 0.7083333333333334, 0.0847457627118644, 0.04166666666666667], [0.30555555555555564, 0.7083333333333334, 0.0847457627118644, 0.04166666666666667], [0.30555555555555564 0.7083333333333334 0.0847457627118644 0.04166666666666667], [0.30555555555555564 0.7083333333333334 0.0847457627118644 0.04166666666666667], 1)

In [18]:
max_elements_small_classes = 5
num_clusters = 10
distance = distances[distanceId]
cId1 =clusterId[distance.ids[1]]
cId2 = clusterId[distance.ids[2]]
if cId1 != cId2
    c1 = clusters[cId1]
    c2 = clusters[cId2]
    count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
    df = DataFrame()
    df."classes" = collect(keys(count_classes_in_clusters))
    df."count_classes" = collect(values(count_classes_in_clusters))
    df = sort(df, [:count_classes], rev=true)
end

Row,classes,count_classes
,Int64,Int64
1,1,2


In [28]:
merge!(c1, c2)

4-element Vector{Float64}:
 0.30555555555555564
 0.7083333333333334
 0.0847457627118644
 0.04166666666666667

In [30]:
mean(c1.x,dims=1)

1×4 Matrix{Float64}:
 0.291667  0.708333  0.0847458  0.0416667

In [33]:
c1.center = mean(c1.x,dims=1)

1×4 Matrix{Float64}:
 0.291667  0.708333  0.0847458  0.0416667

In [34]:
c1.center

1×4 Matrix{Float64}:
 0.291667  0.708333  0.0847458  0.0416667

In [36]:
max_elements_small_classes = 5
num_clusters = 10
distance = distances[distanceId]
cId1 =clusterId[distance.ids[1]]
cId2 = clusterId[distance.ids[2]]
if cId1 != cId2
    c1 = clusters[cId1]
    c2 = clusters[cId2]
    count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
    df = DataFrame()
    df."classes" = collect(keys(count_classes_in_clusters))
    df."count_classes" = collect(values(count_classes_in_clusters))
    df = sort(df, [:count_classes], rev=true)
    # if sum(df[df."count_classes".!=maximum(df."count_classes"),:]."count_classes") <= max_elements_small_classes
    if sum(df[df."classes".!=df[1,1],"count_classes"])<= max_elements_small_classes
        remainingClusters -=1
        
        merge!(c1, c2) #On merge les 2 clusters
        c1.center = mean(c1.x,dims=1)
        for id in c2.dataIds 
            clusterId[id]= cId1 #On modifie le clusterId dans la serie pour le cluster_2, on lui affecte le cluster_1
        end
        # Vider le second cluster
        empty!(clusters[cId2].dataIds)
        push!(empty_clusters, cId2)
    end

    distanceId += 1
end

5

In [25]:
c2.x

1×4 Matrix{Float64}:
 0.166667  0.458333  0.0847458  0.0

In [20]:
clusters[cId1].x

1×4 Matrix{Float64}:
 0.166667  0.458333  0.0847458  0.0

In [17]:
# filter!(e->e≠"s",a)
mean(clusters[1].x,dims=1)

1×4 Matrix{Float64}:
 0.666667  0.541667  0.79661  0.833333